<a href="https://colab.research.google.com/github/AgustinBatistelli/programacion_concurrente/blob/master/TP3/C%2B%2B/TP1_P3_C%2B%2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Voy a usar el comando writefile para escribir 2 archivos. Uno se llamará cliente.cpp y otro se llamará servidor.cpp. Los voy a necesitar a ambos para crear dos ejecutables. Uno para el cliente y otro para el servidor. 

In [1]:
%%writefile cliente.cpp
#include <iostream>
#include <string>
#include <thread>
#include <cstdlib>
#include <ctime>
#include <iomanip>
#include <vector>
#include <ctype.h>
#include <cstring>
#include <regex>
#include <fcntl.h>
#include <sys/stat.h>
#include <unistd.h>

#define EXPRESION_REGULAR "^([1-9][0-9]*|0)-([1-9][0-9]*|0)-([1-9][0-9]*|0)-([1-9][0-9]*|0)$"
#define PARAMETROS_CORRECTOS 2
#define PARAMETROS_INCORRECTOS 0
#define ENTRADA_INCORRECTA PARAMETROS_INCORRECTOS
#define EXITO 1
#define NUMEROS_ENTEROS 4
#define SEPARADOR "-"
#define NOMBRE_FIFO "./mififo"
#define ERROR_FIFO -1
#define INICIALIZADOR 0
#define PRIMERA_POSICION INICIALIZADOR
#define FRACASO EXIT_FAILURE

struct Numero
{
  int numeros[NUMEROS_ENTEROS];
  float promedio;
  int minimo;
  int maximo;
  int suma;
  int cantidad_numeros;
};

bool CantidadCorrectaDeParametros(int parametros_recibidos, int parametros_esperados);
Numero LlenarEstructura(char* entrada);
void MostrarEstructura(Numero numero);

int main(int argc, char* argv[])
{
  if(!CantidadCorrectaDeParametros(argc,PARAMETROS_CORRECTOS))
  {
    std::cout<<"No ingreso la cantidad correcta de parametros. Debe ingresar 4 enteros separados por guion. Ej: 1-2-0-4"<<std::endl;
    return PARAMETROS_INCORRECTOS;
  }
  std::regex regex(EXPRESION_REGULAR);
  if (!std::regex_match(argv[1], regex))
  {
    std::cout<<"La entrada que ingreso NO es correcta"<<std::endl;
    return ENTRADA_INCORRECTA;
  }
  Numero numeros_operar = LlenarEstructura(argv[1]);
  int fifo_fd = open(NOMBRE_FIFO,O_WRONLY);
  if(write(fifo_fd,&numeros_operar,sizeof(Numero))==-1)
  {
    std::cout<<"Error al escribir en el fifo"<<std::endl;
    return FRACASO;
  }
  close(fifo_fd);

  fifo_fd = open(NOMBRE_FIFO,O_RDONLY);
  if(read(fifo_fd,&numeros_operar,sizeof(Numero)) == -1)
  {
    std::cout<<"Error al leer del fifo"<<std::endl;
    return FRACASO;
  }
  close(fifo_fd);
  MostrarEstructura(numeros_operar);
  return EXITO;
}

bool CantidadCorrectaDeParametros(int parametros_recibidos, int parametros_esperados)
{
  return parametros_recibidos == parametros_esperados;
}

Numero LlenarEstructura(char* entrada)
{
  Numero numero;
  numero.cantidad_numeros = numero.maximo = numero.minimo = numero.suma = 0;
  numero.promedio = INICIALIZADOR;
  int i = INICIALIZADOR;
  char* numero_obtenido = std::strtok(entrada, SEPARADOR);
  while (numero_obtenido != nullptr)
  {
    numero.numeros[i++] = std::stoi(numero_obtenido);
    numero_obtenido = std::strtok(nullptr, SEPARADOR);
  }
  return numero;
}

void MostrarEstructura(Numero numero)
{
  std::cout<<"Numeros pasados por parametro: ";
  for (int i = INICIALIZADOR; i < NUMEROS_ENTEROS; i++)
    std::cout<<numero.numeros[i]<<" ";
  std::cout << std::endl;
  std::cout<<"El promedio da: "<<numero.promedio<<std::endl;
  std::cout<<"El minimo es: "<<numero.minimo<<std::endl;
  std::cout<<"El maximo es: "<<numero.maximo<<std::endl;
  std::cout<<"La suma da: "<<numero.suma<<std::endl;
  std::cout<<"La cantidad de numeros es: "<<numero.cantidad_numeros<<std::endl;
}

Writing cliente.cpp


In [2]:
%%writefile servidor.cpp
#include <iostream>
#include <string>
#include <thread>
#include <cstdlib>
#include <ctime>
#include <iomanip>
#include <vector>
#include <ctype.h>
#include <cstring>
#include <regex>
#include <fcntl.h>
#include <sys/stat.h>
#include <unistd.h>

#define EXITO 1
#define FRACASO EXIT_FAILURE
#define NUMEROS_ENTEROS 4
#define NOMBRE_FIFO "./mififo"
#define ERROR_FIFO -1
#define INICIALIZADOR 0
#define PRIMERA_POSICION INICIALIZADOR

struct Numero
{
  int numeros[NUMEROS_ENTEROS];
  float promedio;
  int minimo;
  int maximo;
  int suma;
  int cantidad_numeros;
};

void OperarEstructura(Numero* numero);

int main(int argc, char* argv[])
{
  if (mkfifo(NOMBRE_FIFO, 0600) == ERROR_FIFO)
  {
    std::cout<<"Error al crear el fifo"<<std::endl;
    return FRACASO;
  }
  Numero numeros_para_cliente;
  int fifo_fd = open(NOMBRE_FIFO,O_RDONLY);
  if(read(fifo_fd,&numeros_para_cliente,sizeof(Numero)) == ERROR_FIFO)
  {
    std::cout<<"Error al leer del fifo"<<std::endl;
    return FRACASO;
  }
  close(fifo_fd);
  OperarEstructura(&numeros_para_cliente);
  fifo_fd = open(NOMBRE_FIFO,O_WRONLY);
  if(write(fifo_fd,&numeros_para_cliente,sizeof(Numero)) == ERROR_FIFO)
  { 
    std::cout<<"Error al escribir en el fifo"<<std::endl;
    return FRACASO;
  }
  close(fifo_fd);
  unlink(NOMBRE_FIFO);
  return EXITO;
}

void OperarEstructura(Numero* numero)
{
  for (int i = INICIALIZADOR; i < NUMEROS_ENTEROS; i++)
  {
    numero->suma+=numero->numeros[i];
    numero->cantidad_numeros++;
    if (i == PRIMERA_POSICION)
      numero->minimo = numero->maximo = numero->numeros[i];
    else
    {
      if(numero->numeros[i] < numero->minimo)
        numero->minimo = numero->numeros[i];
      if (numero->numeros[i] > numero->maximo)
        numero->maximo = numero->numeros[i];
    }
  }
  numero->promedio = (float)numero->suma/numero->cantidad_numeros;
}

Writing servidor.cpp


Listo los archivos para asegurarme que ambos archivos .cpp están creados. 

In [3]:
!ls -la

total 24
drwxr-xr-x 1 root root 4096 Apr 28 02:43 .
drwxr-xr-x 1 root root 4096 Apr 28 02:40 ..
-rw-r--r-- 1 root root 3027 Apr 28 02:43 cliente.cpp
drwxr-xr-x 4 root root 4096 Apr 26 17:35 .config
drwxr-xr-x 1 root root 4096 Apr 26 17:36 sample_data
-rw-r--r-- 1 root root 1854 Apr 28 02:43 servidor.cpp


Compilo los dos archivos para generar los ejecutables. Los llamaré Cliente y Servidor. 

In [4]:
!g++ -std=c++11 -o Cliente cliente.cpp

In [5]:
!g++ -std=c++11 -o Servidor servidor.cpp

Vuelvo a listar los archivos para asegurarme de que poseo los dos ejecutables. 

In [6]:
!ls -la

total 612
drwxr-xr-x 1 root root   4096 Apr 28 02:45 .
drwxr-xr-x 1 root root   4096 Apr 28 02:40 ..
-rwxr-xr-x 1 root root 580872 Apr 28 02:45 Cliente
-rw-r--r-- 1 root root   3027 Apr 28 02:43 cliente.cpp
drwxr-xr-x 4 root root   4096 Apr 26 17:35 .config
drwxr-xr-x 1 root root   4096 Apr 26 17:36 sample_data
-rwxr-xr-x 1 root root  20176 Apr 28 02:45 Servidor
-rw-r--r-- 1 root root   1854 Apr 28 02:43 servidor.cpp


Tengo que ejecutar primero el proceso Servidor y luego el cliente. 
Esto es así debido a que le asigné al programa servidor la responsabilidad de crear el archivo FIFO. Por lo tanto, realizaré lo siguiente. 
Ejecutaré el proceso Servidor en segundo plano para poder seguir usando la terminal. Enviaré la salida estandar al archivo salidaC y la salida de error a /dev/null .
Luego de esto, ejecutaré el proceso Cliente. 

In [17]:
!nohup ./Servidor 1>salidaC 2>/dev/null & 

Tras haber puesto al proceso Servidor a correr en segundo plano, ejecutaré el proceso Cliente. Debo ingresarle 4 números enteros separados por guion. Caso contrario, me arrojará un error en el pasaje de parámetros. 
El proceso Cliente deberá: 
 

*   Informar los números pasados por parámetro.
*   Mostrar el promedio de los números.
*   Mostrar el mínimo de los números.
*   Mostrar el máximo de los números.
*   Mostrar la suma de los números.
*   Mostrar la cantidad de números.

In [12]:
!./Cliente 1-2-0-4

Numeros pasados por parametro: 1 2 0 4 
El promedio da: 1.75
El minimo es: 0
El maximo es: 4
La suma da: 7
La cantidad de numeros es: 4


Intento correr el proceso Cliente sin haber ejecutado antes el servidor. Debería dar error al intentar enviar la información al proceso servidor dado a que se comunican mediante un FIFO. Al haber delegado la creación del FIFO al proceso Servidor, este no se ha creado y arrojará error. 

In [13]:
!./Cliente 1-2-0-4

Error al escribir en el fifo


Intento correr el proceso Cliente. Primero sin pasarle ningún parámetro. Luego pasándole dos parámetros. Y por último, pasandole una cadena de entrada no válida. 
Esto para que muestre mensajes de error al validar los parámetros, y comprobar que no está intentando siquiera escribir en el FIFO si los parámetros recibidos no son correctos.

In [14]:
!./Cliente

No ingreso la cantidad correcta de parametros. Debe ingresar 4 enteros separados por guion. Ej: 1-2-0-4


In [15]:
!./Cliente 1-2-0-4 1-8-9-0

No ingreso la cantidad correcta de parametros. Debe ingresar 4 enteros separados por guion. Ej: 1-2-0-4


In [16]:
!./Cliente 1.5-2,6-01-4A

La entrada que ingreso NO es correcta


Voy a ejecutar el proceso Cliente una vez más para comprobar su correcto funcionamiento, esta vez con valores diferentes a los usados en la primera ejecución. Recordar ejecutar el proceso Servidor antes de intentarlo. 

In [18]:
!./Cliente 13-42-10-4

Numeros pasados por parametro: 13 42 10 4 
El promedio da: 17.25
El minimo es: 4
El maximo es: 42
La suma da: 69
La cantidad de numeros es: 4
